In [1]:
from ashlar import fileseries, thumbnail,reg
from ashlar.reg import plot_edge_shifts, plot_edge_quality
from ashlar.reg import BioformatsReader
from ashlar.reg import PyramidWriter
from ashlar.viewer import view_edges
# from ashlar.scripts.ashlar import process_axis_flip
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import argparse
import numpy as np
import pandas as pd
# import pims
import napari
import os
from plotly import express as px
import plotly.io as pio
import kaleido
import skimage
import time
from os.path import abspath
def get_datetime():
    return time.strftime("%Y%m%d_%H%M%S")

In [2]:
def ashlar (files:list, 
        channel_to_align:int, 
        max_shift:int, 
        filter_sigma:int, 
        quiet: bool=False,
        verbose:bool=True,
        output_image_path:str=None
        ):
    
    reader = BioformatsReader(files[0])
    
    if not quiet:
        print("Stitching and registering input images")
        print('Cycle 0:')
        print('    reading %s' % files[0])
    
    aligner = reg.EdgeAligner(reader=reader, channel=channel_to_align, max_shift=max_shift, filter_sigma=filter_sigma, verbose=verbose)
    aligner.run()

    df = pd.DataFrame(
        data = {
            "edge_id": [f"edge_{str(i).zfill(4)}" for i in range(0, aligner.all_errors.shape[0])],
            "cycle_id": "cycle_01",
            'X_Error': np.clip(aligner.all_errors, 0, 10),
            'Y_Shift': np.clip([np.linalg.norm(v[0]) for v in aligner._cache.values()], 0.01, np.inf)
        }
    )

    mosaic_args = {}
    mosaic_args['verbose'] = verbose
    mosaic_args['channels'] = list(range(0, reader.metadata.num_channels))

    mosaics = []
    mosaics.append(reg.Mosaic(aligner, aligner.mosaic_shape, **mosaic_args))

    layers = []

    for i, file in enumerate(files[1:]):
        if not quiet:
            print('    reading %s' % file)
        
        tmp_reader = BioformatsReader(file)
        tmp_aligner = reg.EdgeAligner(reader=tmp_reader, channel=channel_to_align, max_shift=max_shift, filter_sigma=filter_sigma, verbose=verbose)
        tmp_aligner.run()
        tmp_df = pd.DataFrame(
            data = {
                "edge_id": [f"edge_{str(i).zfill(4)}" for i in range(0, tmp_aligner.all_errors.shape[0])],
                "cycle_id": f"cycle_{str(i+2).zfill(2)}",
                'X_Error': np.clip(tmp_aligner.all_errors, 0, 10),
                'Y_Shift': np.clip([np.linalg.norm(v[0]) for v in tmp_aligner._cache.values()], 0.01, np.inf)
            }
        )

        df = pd.concat([df, tmp_df], ignore_index=True)
        
        layer_aligner = reg.LayerAligner(tmp_reader, aligner)
        layer_aligner.run()
        layers.append(layer_aligner)
        mosaics.append(reg.Mosaic(layer_aligner, aligner.mosaic_shape))

    print(f"Writing output image to {output_image_path}")
    writer = PyramidWriter(mosaics, output_image_path, verbose=verbose)
    writer.run()

    return df,aligner,layers

In [3]:
files = [
'/Users/jnimoca/Jose_BI/data/exemplar-001/raw/exemplar-001-cycle-06.ome.tiff',
'/Users/jnimoca/Jose_BI/data/exemplar-001/raw/exemplar-001-cycle-07.ome.tiff',
'/Users/jnimoca/Jose_BI/data/exemplar-001/raw/exemplar-001-cycle-08.ome.tiff'
] 

In [4]:
df, alinger, layers = ashlar(files, channel_to_align=0, max_shift=30, filter_sigma=1, output_image_path='/Users/jnimoca/Jose_BI/data/exemplar-001/ashlar_output.tif')

Stitching and registering input images
Cycle 0:
    reading /Users/jnimoca/Jose_BI/data/exemplar-001/raw/exemplar-001-cycle-06.ome.tiff
    assembling thumbnail 4/6

/opt/homebrew/Caskroom/mambaforge/base/envs/ashlar/lib/python3.10/site-packages/ashlar/reg.py:1367: DataWarning: Stage coordinates' measurement unit is undefined; assuming μm.
  warnings.warn(message, DataWarning)


    assembling thumbnail 6/6
    quantifying alignment error 90/90
    aligning edge 7/7
    reading /Users/jnimoca/Jose_BI/data/exemplar-001/raw/exemplar-001-cycle-07.ome.tiff
    assembling thumbnail 6/6
    quantifying alignment error 20/90

/opt/homebrew/Caskroom/mambaforge/base/envs/ashlar/lib/python3.10/site-packages/ashlar/reg.py:1367: DataWarning: Stage coordinates' measurement unit is undefined; assuming μm.
  warnings.warn(message, DataWarning)


    quantifying alignment error 90/90
    aligning edge 7/7
    assembling thumbnail 6/6
    estimated cycle offset [y x] = [ 1.0646154 45.77846  ]
    reading /Users/jnimoca/Jose_BI/data/exemplar-001/raw/exemplar-001-cycle-08.ome.tiff
    assembling thumbnail 6/6
    quantifying alignment error 20/90

/opt/homebrew/Caskroom/mambaforge/base/envs/ashlar/lib/python3.10/site-packages/ashlar/reg.py:1367: DataWarning: Stage coordinates' measurement unit is undefined; assuming μm.
  warnings.warn(message, DataWarning)


    quantifying alignment error 90/90
    aligning edge 7/7
    assembling thumbnail 6/6
    estimated cycle offset [y x] = [ 0.       38.326153]
Writing output image to /Users/jnimoca/Jose_BI/data/exemplar-001/ashlar_output.tif
Cycle 0:
    Channel 0:
        merging tile 6/6
    Channel 1:
        merging tile 6/6
    Channel 2:
        merging tile 6/6
    Channel 3:
        merging tile 6/6
Cycle 1:
    Channel 0:
    Channel 1:
    Channel 2:
    Channel 3:
Cycle 2:
    Channel 0:
    Channel 1:
    Channel 2:
    Channel 3:
Generating pyramid
    Level 1 (1256 x 1569)
        processing channel 12/12
    Level 2 (628 x 785)
        processing channel 12/12


In [66]:
def plot_scatter_shift_error(df:pd.DataFrame):

    fig = px.scatter(df, x='X_Error', y='Y_Shift', color='cycle_id')

    fig.update_layout(
        title='Error vs Shift', 
        height=800,width=1200,
        paper_bgcolor="white", plot_bgcolor="white",
        uniformtext_minsize=12, uniformtext_mode='hide'
    )

    fig.update_yaxes(
        type="log", nticks=7, 
        showline=True, linecolor="black", mirror=True,
        title=dict(
            text='Shift distance (pixels)',
            font=dict(
                family='Arial',
                size=18,
                color='black'
            )   
        )
    )

    fig.update_xaxes(
        showline=True, linecolor="black", mirror=True,
        title=dict(
            text='Error (NCC)', 
            font=dict(
                family='Arial',
                size=18,
                color='black'
            )   
        )
    )

    fig.update_traces(
        marker=dict(
            size=8, 
            opacity=0.6, 
            line=dict(
                width=1, 
                color='DarkSlateGrey'
            )
        )
    )

    return fig

In [ ]:
def ashlar (files:list, 
        channel_to_align:int, 
        max_shift:int, 
        filter_sigma:int, 
        verbose:bool=True,
        output_image_path:str=None
        ):
    
    reader = BioformatsReader(files[0])
    
    if verbose:
        print("------------------------------------")
        print("Stitching and registering input images")
        print('Cycle 0:')
        print(f'    reading {files[0]}')
    
    aligner = reg.EdgeAligner(reader=reader, channel=channel_to_align, max_shift=max_shift, filter_sigma=filter_sigma, verbose=verbose)
    aligner.run()

    df = pd.DataFrame(
        data = {
            "edge_id": [f"edge_{str(i).zfill(4)}" for i in range(0, aligner.all_errors.shape[0])],
            "cycle_id": "cycle_01",
            'X_Error': np.clip(aligner.all_errors, 0, 10),
            'Y_Shift': np.clip([np.linalg.norm(v[0]) for v in aligner._cache.values()], 0.01, np.inf)
        }
    )

    mosaic_args = {}
    mosaic_args['verbose'] = verbose
    mosaic_args['channels'] = list(range(0, reader.metadata.num_channels))

    mosaics = []
    mosaics.append(reg.Mosaic(aligner, aligner.mosaic_shape, **mosaic_args))

    layers = []

    for i, file in enumerate(files[1:]):
        if verbose:
            print('    reading %s' % file)
        
        tmp_reader = BioformatsReader(file)
        tmp_aligner = reg.EdgeAligner(reader=tmp_reader, channel=channel_to_align, max_shift=max_shift, filter_sigma=filter_sigma, verbose=verbose)
        tmp_aligner.run()
        tmp_df = pd.DataFrame(
            data = {
                "edge_id": [f"edge_{str(i).zfill(4)}" for i in range(0, tmp_aligner.all_errors.shape[0])],
                "cycle_id": f"cycle_{str(i+2).zfill(2)}",
                'X_Error': np.clip(tmp_aligner.all_errors, 0, 10),
                'Y_Shift': np.clip([np.linalg.norm(v[0]) for v in tmp_aligner._cache.values()], 0.01, np.inf)
            }
        )

        df = pd.concat([df, tmp_df], ignore_index=True)
        
        layer_aligner = reg.LayerAligner(tmp_reader, aligner)
        layer_aligner.run()
        layers.append(layer_aligner)
        mosaics.append(reg.Mosaic(layer_aligner, aligner.mosaic_shape))

    print(f"Writing output image to {output_image_path}")
    writer = PyramidWriter(mosaics, output_image_path, verbose=verbose)
    writer.run()

    return df,aligner,layers